# Market Risk - Lab 2

Natacha GAUSSIN

ESILV IF3

## Exercise 2

## Library importation

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

## Dataset

In [2]:
df = pd.read_csv("Lab1_Data.csv", sep = ";")
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
df.sort_values("date", inplace = True)
df["value"] = (df["value"].astype(str).str.replace(",", ".", regex=False))
df["value"] = pd.to_numeric(df["value"], errors="coerce")

df["return"] = df["value"] / df["value"].shift(1) - 1
df.dropna(inplace = True)
df

,date,value,return
1,2015-01-05,5.424,-0.035047
2,2015-01-06,5.329,-0.017515
3,2015-01-07,5.224,-0.019704
4,2015-01-08,5.453,0.043836
5,2015-01-09,5.340,-0.020723
...,...,...,...
1018,2018-12-21,4.045,-0.001481
1019,2018-12-24,4.010,-0.008653
1020,2018-12-27,3.938,-0.017955
1021,2018-12-28,4.088,0.038090


## Parameters

In [3]:
l = 0.94

## Estimation of mu and sigma

In [4]:
mu = np.mean(df["return"])
mu

-9.696604801149598e-05

In [5]:
sum = 0.0
for i in range (0, len(df)):
    sum += df["return"].iloc[i] * df["return"].iloc[i] * np.power(l, len(df) -1 -i)
sigma = np.sqrt((1-l) * sum)
print(sigma*100, "%")

2.424224156705614 %


## Simuler 10000 S1

t = 1 car on cherche à simuler à horizon 1 jour

In [6]:
x = np.random.rand(10000)
#generateur de nombre d'une loi uniforme => nos probas

# on cherche le quantile de la loi normal généré par les proba. Le quantile d'une loi normal 0, 1
q = norm.ppf(x)
print(q)
# on a nos 10000 quantiles

[ 0.59843922 -1.02203316  0.93865365 ... -1.17549204 -0.68397473
 -0.64067876]


In [7]:
# prix à l'horizon t = 1
S = df["value"].iloc[-1] * np.exp(mu - sigma*sigma/2 + sigma*q)
S

array([4.17755904, 4.01662996, 4.21215619, ..., 4.0017151 , 4.04968267,
       4.05393542])

## Compute 10000 call prices

In [8]:
def ComputeC(K, S, r, t, sigma):
    d1 = (np.log(S/K) + (r + sigma*sigma/2)*t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    # calls' price
    C = S*norm.cdf(d1) - K * np.exp(-r*t)*norm.cdf(d2)
    return C

# K is today's price
K = df["value"].iloc[-1]
r = 0
t = 30
C = ComputeC(K, S, r, t, sigma)
print(C)


# call à t = 0 donc S0 = K
C0 = ComputeC(K=K, S=K, r=r, t=t+1, sigma=sigma)
print(C0)

[0.25009736 0.16798594 0.27018912 ... 0.161342   0.18329783 0.18532674]
0.22162865387169073


## Compute the return of C0 and C

In [9]:
C_return = C/C0 - 1

In [10]:
print(C_return)

[ 0.12845226 -0.24203872  0.21910735 ... -0.27201652 -0.17295069
 -0.16379614]


## Compute Value at Risk

In [11]:
alpha = 0.95

def Var(C, alpha):
    C = sorted(C)
    return C[int(len(C)*(1 - alpha))]

Var95 = Var(C_return, alpha)
print(Var95)

-0.3557377899900761
